# generate option rst files

In [1]:
import pandas as pd
# load processed csv of df_state info
df_var_info = pd.read_csv('./df_var_supy.csv')


FileNotFoundError: File b'./df_var_supy.csv' does not exist

In [0]:
# generate option string for rst option file
def gen_opt_str(ser_rec):

    name = ser_rec['variable']
    desc = ser_rec['Description']
    vars = ser_rec['SUEWS-related variables']
    dim = ser_rec['Dimensionality']
    remark = ser_rec['Dimensionality Remarks']
    str_opt = '''
.. option:: {name}

    :Description:
        {desc}
    :SUEWS-related variables:
        {vars}
    :Dimensionality:
        {dim}
    :Dimensionality remarks:
        {remark}'''.format(
        name=name,
        desc=desc,
        vars=vars,
        dim=dim,
        remark=remark,
    )
    return str_opt


gen_opt_str(df_var_info.iloc[10])


In [0]:

df_var_info['rst'] = df_var_info.apply(gen_opt_str, axis=1)
df_var_info = df_var_info.sort_values('variable').reset_index(drop=True)
rst_txt = '\n\n'.join(df_var_info.rst)
# print()
with open('df_state.rst', 'w') as f:
    print(rst_txt, file=f)

df_var_info.tail()

